# Задание 7

## 1. PropertyCreator (0.2 балла)

Напишите мета класс для создания свойств (property) класса из функций начинающихся с "set\_", "get\_" или "del_". Пример использования:
<code>
class TestPropertyCreator(metaclass=PropertyCreator):
    def \__init\__(self, lo):
        self.__x = None
        self.lo = lo

    def get_x(self):
        return self.__x

    def set_x(self, value):
        if value < self.lo:
            raise ValueError("Value must in condition: {} <= value".format(self.lo))
        self.__x = value
    
    def del_x(self):
        self.__x = "No more"

    pass


obj = TestPropertyCreator(5)
obj.x = 4
print(obj.x)
del (obj.x)
</code>

* Мета класс должен поддерживать наследование, в смысле создавать свойства у потомков.
* Должен поддерживать частичное описание свойств, т. е. например, описание одного метода get_val (без set_val и del_val).
* Поддерживать множественное использование одного свойства с одним именем в разных классах:

<code>
class A(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class A"

class B(metaclass=PropertyCreator):
    def get_x(self):
        return "x in class B"

class C(metaclass=PropertyCreator):
    def set_x(self, value):
        self.value = "x in class C"
    def get_x(self):
        return self.value
</code>
* Должен уметь обрабатывать имен с несколькими подчеркиваниями "get_raw_text".

#### Решение

In [1]:
import re

In [2]:
dict_ = dict()

In [3]:
class PropertyCreator(type):
    
    def __new__(cls, name, bases, attrs):

        properties = dict()
        
        def add_property(pr_name, pr_type, pr_body):
            if (properties.get(pr_name) == None):
                properties[pr_name] = {}
            properties[pr_name][pr_type] = pr_body
        
        for attr_name, attr_body in attrs.items():
            if (callable(attr_body)):
                if (attr_name.startswith('get_') or attr_name.startswith('set_') or
                    attr_name.startswith('del_')):
                    add_property(attr_name[4:], attr_name[:3], attr_body)
         
        for prop_name in properties:
            attrs[prop_name] = property(properties[prop_name].get('get'), properties[prop_name].get('set'), 
                                       properties[prop_name].get('del'))
            
        return super().__new__(cls, name, bases, attrs)

In [4]:
cls = PropertyCreator('Hello', (object, ), {})
get_pat = re.compile('get_')

In [5]:
class SimplePropertyCreator(metaclass=PropertyCreator):
        def __init__(self, lo):
            self.__x = None
            self.lo = lo
            
        def get_x(self):
            return self.__x
        
        def get_get_x(self):
            return 0

        def set_x(self, value):
            print("i'm in set")
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"

In [6]:
testingClass = SimplePropertyCreator(4)

In [7]:
testingClass.x = 5
print(testingClass.x)

i'm in set
5


In [299]:
simpleTest = TestSimple(5)
# .test_get_property()
simpleTest.test_get_property()

#### Протестируйте свое решение

In [ ]:
def test_simple():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def init(self, lo):
            self.__x = None
            self.lo = lo
            
        def get_x(self):
            return self.__x

        def set_x(self, value):
            if value < self.lo:
                raise ValueError("Value must in condition: {} <= value".format(self.lo))
            self.__x = value

        def del_x(self):
            self.__x = "No more"
    <your code here>

    
def test_with_inheritance():
    class TestPropertyCreator(metaclass=PropertyCreator):
        pass

    class TestPropertyCreatorInheritance(A):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_x(self, value):
            self._secret_list.append("set")
    <your code here>

    
def test_partially_defined():
    class TestPropertyCreator(metaclass=PropertyCreator):
        def __init__(self):
            self._secret_list = []

        def get_x(self):
            self._secret_list.append("get")
            return 0

        def set_y(self, value):
            self._secret_list.append("set")
            self._y = value
    <your code here>

    
def test_sanity():
    class TestPropertyCreator(metaclass=PropertyCreator):
        _text = 0
        def get_raw_text(self):
             return self._boo

        def get_text(self):
             return self._text % 2

        def set_text(self, value):
            try:
                self._text = int(value)
            except ValueError:
                raise TypeError("unproper value for text: {}".format(value))
    <your code here>


def test_multiple_usages():
    class TestPropertyCreatorA(metaclass=PropertyCreator):
        def get_x(self):
            return 0
    class TestPropertyCreatorB(metaclass=PropertyCreator):
        def get_x(self):
            return 1
    class TestPropertyCreatorC(metaclass=PropertyCreator):
        def set_x(self, value):
            self.value = value + 1
        def get_x(self):
            return self.value
    <your code here>

## 2. InstanceCountExeptioner (0.2 балла)
Напишите метакласс InstanceCountExeptioner, который будет следить за количеством экземпляров класса, использующих его. Количество задается через поле класса \_\_max_instane\_count\_\_. Т. е. число экземпляров каждого класса регулируется отдельно. Если в классе не указано поле \_\_max_instane\_count\_\_, то используйте заранее заданное число в метаклассе (любое). Пример:

<code>
class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 2
    def \__init\__(self, a):
        self.a = a


class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    \_\_max_instane\_count\_\_ = 1
    def \__init\__(self, a):
        self.a = a

a_one = TestInstanceCountExeptionerA('one')
a_two = TestInstanceCountExeptionerA('two')
b_one = TestInstanceCountExeptionerB('one')
\# пока всё шло хорошо

\# а вот
a_three = TestInstanceCountExeptionerA('three')
\# выкенет исключение InstanceCountExeption (ваше собственное исключение)
</code>

In [9]:
try:
    1/0
except Exception as e:
    print(e)

division by zero


#### Решение

In [10]:
# Опишите исключение InstanceCountExeption
class InstanceCountExeption(Exception):
    def __str__(self):
        return "Too many insatancies"

In [125]:
# Опишите мета класс InstanceCountExeptioner
class InstanceCountExeptioner(type):

    __current_instance_counter__ = 0
        
    def __call__(cls, *args, **kwargs):
        cls.__current_instance_counter__ += 1
        
        if (cls.__current_instance_counter__ <= cls.__max_instance_count__):
            return super().__call__(*args, **kwargs)
        else:
            raise InstanceCountExeption

#### Протестируйте свое решение

In [129]:
class TestInstanceCountExeptionerA(metaclass=InstanceCountExeptioner):
    __max_instance_count__ = 2

    def __init__(self, counter):
        self.a = 1

    def get(self):
        return self.a

In [130]:
class TestInstanceCountExeptionerB(metaclass=InstanceCountExeptioner):
    __max_instance_count__ = 3

    def __init__(self, counter):
        self.b = 2

    def get(self):
        return self.b

In [131]:
a_one = TestInstanceCountExeptionerA('one')
a_two = TestInstanceCountExeptionerA('two')
b_one = TestInstanceCountExeptionerB('one')
# пока всё шло хорошо

1 2
2 2
1 3


In [132]:
# а вот
a_three = TestInstanceCountExeptionerA('three')
# выкенет исключение InstanceCountExeption (ваше собственное исключение)

3 2


InstanceCountExeption: Too many insatancies

In [133]:
    
def test_simple(self):
    <your code here>
    

def test_create(self):
    <your code here>

def test_fail_create_a(self):
    <your code here>
    try:
        <your code here>
    except InstanceCountExeption as e:
        <your code here>
    

def test_fail_create_b(self):
    <your code here>
    try:
        <your code here>
    except InstanceCountExeption as e:
        <your code here>

IndentationError: expected an indented block (<ipython-input-133-250157564a31>, line 3)

## 3. JSONClassCreator (0.6 баллов)
Напишите метакласс, который будет по json представлению строить новый класс и обратно. Класс должен уметь следующее:
* Поддерживать сохранение и получение магических функций класса.
* Поддерживать сохранение и получение обычных функций.
* Поддерживать сохранение полей со стандартными типами.
* Уберите из сохранения следующие поля и методы: ['\_\_dict\_\_', '\_\_weakref\_\_', '\_\_module\_\_', '\_\_init\_\_']
* У создаваемого класса должна быть функция to_json_str

Формат json строки должен быть следующий:

<code>
{
    "name": название класса,
    "bases": базовые классы,
    "methods": методы класса,
    "attrs": поля класса
}
</code>

Рекомендации:
* Для получения кода функций используйте модуль <a href="http://python-lab.ru/documentation/27/stdlib/inspect.html">inspect</a>.
* Для того, чтобы запустить код функций, можно использовать exec.
* Можно не исправлять ошибку типа OSError: could not get source code - возникает для функций, полученных с помощью exec.

#### Пример использования

In [2]:
import json


class ParentTest1(object):
    pass

class ParentTest2(object):
    pass

class Test(ParentTest1, ParentTest2):
    """Тестовый класс"""

    val = [1, 2, 3]

    def f(self, x):
        print(x)
    
    def __repr__(self):
        return "Test(val={})".format(self.val)

    def __str__(self):
        return "Test(val={})".format(self.val)

    pass

In [5]:
print(*json.loads(JSONClassCreator.to_json_str(Test)).items(), sep="\n")

('name', 'Test')
('bases', ['ParentTest1', 'ParentTest2'])
('methods', {'f': 'def f(self, x):\n        print(x)', '__repr__': 'def __repr__(self):\n        return "Test(val={})".format(self.val)', '__str__': 'def __str__(self):\n        return "Test(val={})".format(self.val)'})
('attrs', {'__doc__': 'Тестовый класс', 'val': [1, 2, 3]})


In [6]:
tmp = JSONClassCreator(JSONClassCreator.to_json_str(Test))

In [7]:
tmp_obj = tmp()
tmp_obj, tmp_obj.f("hi"), tmp.val, tmp.__doc__

hi


(Test(val=[1, 2, 3]), None, [1, 2, 3], 'Тестовый класс')

In [10]:
tmp.__dict__

mappingproxy({'__doc__': 'Тестовый класс',
              '__module__': '__main__',
              '__repr__': <function __main__.__repr__>,
              '__str__': <function __main__.__str__>,
              'f': <function __main__.f>,
              'to_json_str': <function __main__.JSONClassCreator.to_json_str>,
              'val': [1, 2, 3]})

#### Решение

In [ ]:
import inspect


class JSONClassCreator(type):
    def __new__(mcls, json_str):
        <your code here>

    def to_json_str(cls):
        exclude = ['__dict__', '__weakref__', '__module__', '__init__']
        <your code here>

        return json.dumps({
            "name": <your code here>,
            "bases": <your code here>,
            "methods": <your code here>,
            "attrs": <your code here>
        })

    pass


#### Проверьте свое решение на примере